In [30]:
import os
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import scipy as sp
import sensuous.preprocessing.csv as prep
from sensuous.model.knn import strip_artists, predict_playlist_csv
from sklearn.preprocessing import MinMaxScaler

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [34]:
prep.preprocess_csv_data()

Csv data preprocessing done, saved the cleaned data to file "../data/all-songs.csv"


In [4]:
df = pd.read_csv('../data/all-songs.csv')

In [5]:
df = df.drop(columns='Unnamed: 0')

In [14]:
n_neighbors = 10

In [28]:
df_numeric = df.drop(columns=['artists', 'song_title'])
columns = df_numeric.columns.values.tolist()
df_scaler = MinMaxScaler()
df[columns] = df_scaler.fit_transform(df[columns])

# Seed song selection
song_title = 'I Miss You'
artist = 'Teddy Pendergrass'

criterion = ((df['artists'].map(lambda x: artist in x) 
              | df['song_title'].map(lambda x: 'feat. ' + artist in x))
             & (df['song_title'].map(lambda x: song_title in x)))
seed_song = df[criterion]

if seed_song.shape[0] == 0:
    print("No such a song in the dataset. Returning the user's entry.")
#     return f'{song_title} by {artist}'
elif seed_song.shape[0] > 1:
    print("There is more than one entry. Returning the search results.")
#     return seed_song

# Feature "preprocessing"
seed_song_features = seed_song.select_dtypes(exclude='object')
X = df.select_dtypes(exclude='object')

# List of closest neighbors to the seed song
tree = sp.spatial.KDTree(X.to_numpy())
distances, indeces = tree.query([seed_song_features], k=n_neighbors)
indeces = indeces.flatten()
neighbors_df = df.loc[list(indeces)]
songs = neighbors_df['song_title'].tolist()
artists = [
    strip_artists(artist) for artist in neighbors_df['artists'].tolist()
]
playlist = {'playlist': list(zip(songs, artists))}

In [31]:
song_title = 'I Miss You'
artist = 'Teddy Pendergrass'

In [36]:
predict_playlist_csv(song_title, artist, data_dir='../data')

{'playlist': [('I Miss You (feat. Teddy Pendergrass)',
   'Harold Melvin & The Blue Notes, Teddy Pendergrass'),
  ('Tumse Milkar Na Jane', 'Lata Mangeshkar, Shabbir Kumar'),
  ('The Puppet', 'Echo & the Bunnymen'),
  ('Majhe Mahar Pandhari - Abhang 1977', 'Bhimsen Joshi'),
  ('I Am The Walrus - Remastered 2009', 'The Beatles'),
  ('Mystic Traveler', 'Dave Mason'),
  ('Rat Bat Blue', 'Deep Purple'),
  ('Hola les lolos', 'Bernard Adamus'),
  ('Gori Tera Gaon Bada Pyara', 'K. J. Yesudas'),
  ('I Choose You', 'Willie Hutch')]}

In [1]:
os.getcwd()


NameError: name 'os' is not defined

In [2]:
import os


In [3]:
os.getcwd()


'/Users/sergey/code/prujaka/spotify-sensuous-backend/notebooks'